# Báo cáo đồ án môn "Lập trình song song ứng dụng"

Nhóm 5:
1. 1712718 - Huỳnh Thanh Sang - [GitHub](https://github.com/hts7117)
2. 1712683 - Phạm Hoàng Phương - [Github](https://github.com/superman19993)
3. 1712584 - Nguyễn Công Lý - [GitHub](https://github.com/conglyne222)

Link thùng chứa Github của nhóm: [Repo](https://github.com/superman19993/LTSSUD_1712718_1712683_1712584)


## Mô tả ứng dụng
  ### Object detection and Segmentation by Mask RCNN
  - Input: Ảnh đầu vào có kích thước W x H ![image.png](https://scontent.fsgn5-9.fna.fbcdn.net/v/t1.15752-9/277380635_662833644832542_6356397547685485335_n.jpg?_nc_cat=105&ccb=1-5&_nc_sid=ae9488&_nc_ohc=jB9jbjeLC10AX-Yzd47&_nc_ht=scontent.fsgn5-9.fna&oh=03_AVL57_7pZvCV8dNWX_QjrJck_jP1h1AJ8Y9pYSLfPnc6-Q&oe=626ED38B)
  - Output: Các bounding box:[cx, cy, w, h] (tọa độ tâm + kích thước) tại các vị trí nghi ngờ có cá thể ![image.png](https://scontent.fsgn5-13.fna.fbcdn.net/v/t1.15752-9/277262287_1207776213380954_9024006818679567261_n.jpg?_nc_cat=104&ccb=1-5&_nc_sid=ae9488&_nc_ohc=poGvM4_xUf0AX8uv1Ym&_nc_ht=scontent.fsgn5-13.fna&oh=03_AVLbmcpGtehwLCGbdCKkpenj4FN3iOQGcz1-qGGB-HwX9A&oe=626BD5BA)

  - Framework của bài toán: ![image.png](https://scontent.fsgn5-5.fna.fbcdn.net/v/t1.15752-9/276021156_548651876683254_6016110322686193894_n.png?_nc_cat=100&ccb=1-5&_nc_sid=ae9488&_nc_ohc=ckXISxUADFYAX_pDEvq&_nc_ht=scontent.fsgn5-5.fna&oh=03_AVJN_vkl_ifJCDKZqRY4bczOy-tb4S8YPSDgP9DNSGF-Mw&oe=626BBCFF)
  - Trong đề tài này, nhóm 5 lấy mô hình object detection and segmentation by Mask RCNN đã được huấn luyện từ trước để thực hiện tối ưu về thời gian chạy.
  - Mô hình trên được huấn luyện từ tập dữ liệu COCO (200.000 ảnh, 80 lớp, 1.500.000 cá thể).




  ### Lý do chọn đề tài: 
  - Bài toán sử dụng mạng CNN (liên quan đến tác vụ convolution trên ảnh).

## Kiến trúc Mask-RCNN


![image.png](https://scontent-hkg4-2.xx.fbcdn.net/v/t1.15752-9/278021850_714827902981298_8096012180834245696_n.png?_nc_cat=109&ccb=1-5&_nc_sid=ae9488&_nc_ohc=E4Y24vI4HUoAX8buWt7&tn=ZPvUmtLj3aIjaptG&_nc_ht=scontent-hkg4-2.xx&oh=03_AVKM7EvMqKuoSM00tjZqi0quhHjfyw_eo4gXUFRODZkjnA&oe=6283B58C)
- Maximum size ảnh là 800 x 1333
- Các chỉ số width và height cần phải chia hết cho 32 vì ở feature map được sinh ra ở layer cuối cùng của FPN có kích thước là w/32, h/32.

### Phát sinh ứng viên (Region Proposal)
- Đầu tiên các ảnh input đầu vào được đưa qua mạng rút trích đặc trưng bằng 1 trong các cách Feature Pyramid Network( FPN) để rút ra các feature map. Kích thước của feature map nhỏ hơn rất nhiều so với ảnh ban đầu, bù lại số channel tăng lên nên vẫn bảo toàn được thông tin cấu trúc ảnh.
- Sau đó sẽ duyệt trên từng ô của feature map một lần duy nhất, tại mỗi ô sẽ sinh ra k anchor box có kích thước cố định. Cách sinh anchor box này đảm bảo mô hình có thể nhận dạng được các cá thể thuộc mọi kích thước và hình dạng khác nhau.
- Minh họa anchor box k=9
![image.png](https://scontent.xx.fbcdn.net/v/t1.15752-9/277967659_2116324165207464_498970705263918862_n.png?_nc_cat=104&ccb=1-5&_nc_sid=aee45a&_nc_ohc=cWy5fHPVNbQAX8CY7ST&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.xx&oh=03_AVJbspaszyA_gIFUTo7K9bOHaBzYPkXn7lLQqAQVchLLWA&oe=628762EA)


- Feature map đang xét sau đó được đưa vào RPN( Region Proposal Network) để kiểm tra từng anchor box có chứa vật thể không và tinh chỉnh anchor box đó. Những anchor box có chứa vật thể sẽ được làm bounding box ứng viên

### Phân lớp (Classification)
- Sau đó các bounding box ứng viên sẽ được đưa qua ROI Align để crop các feature map ở các ứng viên ra.
- Phần feature map sau khi thực hiện ROIAlign sẽ được đưa vô một mạng fully-connected gồm 2 nhánh để tìm ra class id tương ứng với bounding box đầu vào và tinh chỉnh lại bounding box đầu vào để tăng độ chính xác. 

### Phân đoạn ảnh (Segmentation)
- Với bounding box được tinh chỉnh ở phần phân lớp, chúng ta lại crop feature map từ bước rút trích đặc trưng và cho qua ROIAlign. Phần feature map kết quả sẽ được cho qua một 1 mạng Convolution với output có kích thước tổng số class x 28 x 28. Cuối cùng, lấy output thứ class_id (kết quả phân lớp) và resize lại cho kích thước ban đầu của bounding box ta được binary mask cần tìm.

## Mask R-CNN inference stage
A quick intro to using the pre-trained model to detect and segment objects.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/LTSS_UD/

/content/gdrive/MyDrive/LTSS_UD


In [ ]:
%tensorflow_version 1.14.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
!cp saving.py /tensorflow-1.15.2/python3.7/keras/engine

In [ ]:
!python testing.py

Using TensorFlow backend.


2022-04-17 02:21:05.169431: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-04-17 02:21:05.169623: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555a73ef3800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-04-17 02:21:05.169654: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-04-17 02:21:05.171237: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-04-17 02:21:05.400114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 02:21:05.400857: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555a73ef3100 initialized for platform CUDA (this does not guarantee that XLA will 